In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
import json

import jax
from jax import numpy as jnp, jit, value_and_grad, Array
from jax.typing import ArrayLike
import flax
from flax import linen as nn
from flax.training import train_state
from flax.core import FrozenDict
import optax

### Data Preproscessing

In [2]:
metadata = pd.read_csv('musicnet/musicnet_metadata.csv')
data_files = glob('musicnet/musicnet/*/*.wav')

In [3]:
n_mels = 512

def wav_to_mel_spec(path: str) -> np.ndarray:
  y, sr = librosa.load(path)
  spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
  return np.abs(librosa.amplitude_to_db(spec, ref=np.max))

data = [wav_to_mel_spec(path) for path in data_files]

In [4]:
data_ids = [int(path[-8:-4]) for path in data_files]
labels = [metadata[metadata['id'] == i]['ensemble'].values[0] for i in data_ids]

In [5]:
labels_to_nums = {label: i for i, label in enumerate(sorted(set(labels)))}
nums_to_labels = {i: label for label, i in labels_to_nums.items()}

labels = [labels_to_nums[label] for label in labels]

In [6]:
new_data = []
new_labels = []

sample_len = 512

for x, y in zip(data, labels):
  for i in range(0, x.shape[1] - sample_len + 1, sample_len):
    new_data.append(np.expand_dims(x[:, i:i + sample_len], axis=2))
    new_labels.append(y)

In [7]:
x_full = np.array(new_data, np.float32)
y_full = np.array(new_labels, np.int32)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_full, y_full, test_size=0.25, random_state=0)

### Model

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 16
train_ds = train_ds.shuffle(buffer_size=len(x_train)).batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [10]:
class CNN(nn.Module):
  @nn.compact
  def __call__(self, x: ArrayLike, training: bool) -> Array:
    x = (nn.Conv(features=8, kernel_size=(3, 3), use_bias=False))(x)
    x = nn.BatchNorm(use_running_average=not training)(x)
    x = nn.relu(x)
    x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))

    x = nn.Conv(features=8, kernel_size=(3, 3), use_bias=False)(x)
    x = nn.BatchNorm(use_running_average=not training)(x)
    x = nn.relu(x)
    x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))

    x = nn.Conv(features=8, kernel_size=(3, 3), use_bias=False)(x)
    x = nn.BatchNorm(use_running_average=not training)(x)
    x = nn.relu(x)
    x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))

    x = x.reshape((x.shape[0], -1))
    
    x = nn.Dense(features=128)(x)
    x = nn.relu(x)

    x = nn.Dense(features=64)(x)
    x = nn.relu(x)

    x = nn.Dense(features=21)(x)
    return x

In [11]:
class TrainState(train_state.TrainState):
  batch_stats: FrozenDict

def create_train_state(model: CNN, rng_key: Array, learning_rate: float) -> TrainState:
  variables = model.init(rng_key, x=jnp.ones((1, *x_train.shape[1:])), training=False)
  return TrainState.create(
    apply_fn=model.apply, 
    params=variables['params'],
    batch_stats=variables['batch_stats'], 
    tx=optax.adamw(learning_rate, weight_decay=2e-3),
  )

In [12]:
@jit
def train_step(state: TrainState, batch: ArrayLike) -> TrainState:
  
  def loss_fn(params: FrozenDict) -> tuple[Array, FrozenDict]:
    logits, updates = state.apply_fn({
      'params': params, 
      'batch_stats': state.batch_stats,
    }, x=batch[0], training=True, mutable=['batch_stats'])

    loss = optax.softmax_cross_entropy_with_integer_labels(logits=logits, labels=batch[1]).mean()
    return loss, updates
  
  grad_fn = value_and_grad(loss_fn, has_aux=True)
  (_, updates), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  state = state.replace(batch_stats=updates['batch_stats'])
  return state

In [13]:
@jit
def accuracy(state: TrainState, batch: ArrayLike):
  logits = state.apply_fn({
    'params': state.params, 
    'batch_stats': state.batch_stats,
  }, x=batch[0], training=False)
  
  preds = jnp.argmax(logits, axis=1)
  acc = jnp.mean(preds == batch[1])
  return acc

In [14]:
flax_model = CNN()
state = create_train_state(flax_model, jax.random.PRNGKey(0), learning_rate=1e-4)

2023-12-23 20:37:12.821952: W pjrt_plugin/src/mps_client.cc:534] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


Metal device set to: Apple M3 Pro

systemMemory: 18.00 GB
maxCacheSize: 6.00 GB



In [15]:
num_epochs = 15

for epoch in range(num_epochs):
  for batch in train_ds.as_numpy_iterator():
    state = train_step(state, batch)
  
  train_acc_list = []
  test_acc_list = []

  for batch in train_ds.as_numpy_iterator():
    acc = accuracy(state, batch)
    train_acc_list.append(acc)

  for batch in test_ds.as_numpy_iterator():
    acc = accuracy(state, batch)
    test_acc_list.append(acc)
  
  train_acc = sum(train_acc_list) / len(train_acc_list)
  test_acc = sum(test_acc_list) / len(test_acc_list)

  print(f'[epoch {epoch + 1}] train acc: {train_acc}, test acc: {test_acc}')

[epoch 1] train acc: 0.7633345127105713, test acc: 0.7220911979675293
[epoch 2] train acc: 0.7240667343139648, test acc: 0.6623427867889404
[epoch 3] train acc: 0.8015642762184143, test acc: 0.7197327017784119
[epoch 4] train acc: 0.9355345964431763, test acc: 0.8132861852645874
[epoch 5] train acc: 0.9849318861961365, test acc: 0.8419811129570007
[epoch 6] train acc: 0.990435004234314, test acc: 0.8525943160057068
[epoch 7] train acc: 0.9962002038955688, test acc: 0.8620283007621765
[epoch 8] train acc: 0.9854559898376465, test acc: 0.8392295837402344
[epoch 9] train acc: 0.99947589635849, test acc: 0.8820754885673523
[epoch 10] train acc: 0.9993448853492737, test acc: 0.8651729822158813
[epoch 11] train acc: 0.9997379183769226, test acc: 0.8860062956809998
[epoch 12] train acc: 0.9998689889907837, test acc: 0.885613203048706
[epoch 13] train acc: 0.9998689889907837, test acc: 0.8706761002540588
[epoch 14] train acc: 0.9975104928016663, test acc: 0.8694968819618225
[epoch 15] train ac

In [16]:
params_save_path = 'checkpoints/params.json'
params_dict = {k1: {k2: v2.tolist() for k2, v2 in v1.items()} for k1, v1 in state.params.items()}
with open(params_save_path, 'w') as f:
  json.dump(params_dict, f)

batch_stats_save_path = 'checkpoints/batch_stats.json'
batch_stats_dict = {k1: {k2: v2.tolist() for k2, v2 in v1.items()} for k1, v1 in state.batch_stats.items()}
with open(batch_stats_save_path, 'w') as f:
  json.dump(batch_stats_dict, f)